In [55]:
import os
from dotenv import load_dotenv

In [56]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")

In [57]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [58]:
arxiv_api = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_api = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)

In [59]:
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)
wikipedia = WikipediaQueryRun(api_wrapper=wiki_api)

In [60]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

In [61]:
llm = ChatGroq(model="qwen-qwq-32b")

In [62]:
loader = WebBaseLoader("https://js.langchain.com/docs/introduction/")
docs = loader.load()

In [63]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = splitter.split_documents(docs)

In [64]:
embeddings = MistralAIEmbeddings()

c:\Users\nikhil\.conda\envs\rag\lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [65]:
vector = FAISS.from_documents(documents, embeddings)
query = "What is LangChain?"
docs = vector.similarity_search(query, k=1)
print(docs[0].page_content)

framework for developing applications powered by large language models (LLMs).LangChain simplifies every stage of the LLM application lifecycle:Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations.


In [66]:
retriever = vector.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'MistralAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000258B2F20220>, search_kwargs={})

In [67]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=retriever, name="LangChain Docs", description="LangChain documentation")

In [68]:
retriever_tool.name

'LangChain Docs'

In [69]:
tools = [arxiv, wikipedia, retriever_tool]

In [70]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\nikhil\.conda\envs\rag\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [71]:
from langchain.agents import create_openai_tools_agent
# from langchain_community.agents import create_groq_tools_agent
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

In [72]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [73]:
agent_executor.invoke({"input" : "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `LangChain Docs` with `{'query': 'what is langchain'}`


framework for developing applications powered by large language models (LLMs).LangChain simplifies every stage of the LLM application lifecycle:Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations.

Use LangGraph.js to build stateful agents with first-class streaming and human-in-the-loop support.Productionization: Use LangSmith to inspect, monitor and evaluate your chains, so that you can continuously optimize and deploy with confidence.Deployment: Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Cloud.Concretely, the framework consists of the following open-source libraries:@langchain/core: Base abstractions and LangChain Expression Language.@langchain/community: Third party integrations.Partner packages (e.g. @langchain/openai, @langchain/anthropic, etc.): Some inte

{'input': 'what is langchain?',
 'output': 'LangChain is a **development framework** designed to simplify building applications powered by large language models (LLMs). It provides tools and abstractions to streamline the entire lifecycle of LLM-driven applications, from prototyping to deployment. Here\'s a concise breakdown:\n\n### Core Purpose\n- **Simplifies LLM Application Development**: Offers components for building, testing, and deploying AI apps that leverage LLMs like GPT, Claude, etc.\n- **Focus Areas**:\n  1. **Development**: Provides modular building blocks (e.g., chains, agents, retrievers) for constructing AI workflows.\n  2. **Productionization**: Enables monitoring, evaluation, and optimization via tools like **LangSmith** (for tracing and analysis).\n  3. **Deployment**: Supports scaling apps into production with state management and APIs (e.g., via **LangGraph.js** for interactive workflows).\n\n### Key Components\n- **@langchain/core**: Foundation with core abstracti